In [1]:
import pandas as pd
import numpy as np
import miceforest as mf


In [2]:
df = pd.read_csv("../data/V2_TEST_DATA_WITHDEMOGRAPHICS.csv")

In [3]:
df = df.drop(columns=["Unnamed: 0", "Unnamed: 0.1"])

In [4]:
df.head()

,CrimeDate,Description,Post,District,Neighborhood,Longitude,Latitude,Premise,Month,Hour,...,perc18_24,perc25_64,perc65up,perc_asian,perc_aa,perc_hisp,perc_white,median_price_homes_sold,racial_diversity_index,num_households
0,2018-12-30,7,523.0,3,HOMELAND,39.3556,-76.6099,78,12,0,...,25.103793,43.962919,15.253370,10.805892,15.947222,3.469260,64.852414,338350.0,54.141457,347500.0
1,2018-12-29,4,123.0,0,UPTON,39.3011,-76.6276,69,12,0,...,10.168599,45.584826,12.465753,1.401475,90.769231,1.064278,4.573235,105500.0,18.546981,53465.0
2,2018-12-29,12,441.0,2,PULASKI INDUSTRIAL AREA,39.3039,-76.5408,38,12,0,...,5.425101,50.024291,11.303644,1.943320,31.433198,26.542510,38.801619,125000.0,77.658458,83000.0
3,2018-12-28,4,942.0,6,RIVERSIDE,39.2736,-76.6118,50,12,0,...,4.993910,77.425903,6.645013,2.287184,2.354852,3.315740,90.634727,350000.0,20.090743,306000.0
4,2018-12-28,3,613.0,4,LANGSTON HUGHES,39.3438,-76.6777,78,12,0,...,9.433962,49.834663,24.129547,0.437658,94.728652,0.816962,2.655125,69680.0,11.410513,40000.0


In [5]:
df.columns


Index(['CrimeDate', 'Description', 'Post', 'District', 'Neighborhood',
       'Longitude', 'Latitude', 'Premise', 'Month', 'Hour', 'Year', 'Outside',
       'Season_autumn', 'Season_spring', 'Season_summer', 'Season_winter',
       'Weapon_FIREARM', 'Weapon_HANDS', 'Weapon_KNIFE', 'Weapon_NONE',
       'Weapon_OTHER', 'Holiday', 'Weekend', 'CSA', 'median_household_income',
       'households_below_poverty', 'perc18_24', 'perc25_64', 'perc65up',
       'perc_asian', 'perc_aa', 'perc_hisp', 'perc_white',
       'median_price_homes_sold', 'racial_diversity_index', 'num_households'],
      dtype='object')

In [6]:
df = df.drop(columns=["CrimeDate","CSA"])


In [7]:
df.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 51119 entries, 0 to 51118
Data columns (total 34 columns):
 #   Column                    Non-Null Count  Dtype  
---  ------                    --------------  -----  
 0   Description               51119 non-null  int64  
 1   Post                      51119 non-null  float64
 2   District                  51119 non-null  int64  
 3   Neighborhood              51119 non-null  object 
 4   Longitude                 51119 non-null  float64
 5   Latitude                  51119 non-null  float64
 6   Premise                   51119 non-null  int64  
 7   Month                     51119 non-null  int64  
 8   Hour                      51119 non-null  int64  
 9   Year                      51119 non-null  int64  
 10  Outside                   51119 non-null  int64  
 11  Season_autumn             51119 non-null  int64  
 12  Season_spring             51119 non-null  int64  
 13  Season_summer             51119 non-null  int64  
 14  Season

Wonder if this will work with the saved imputer...


In [8]:
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()

In [9]:
Neighborhood = pd.read_csv("../data/V2_Neighborhood_decoder.csv", header=None)
le.fit(Neighborhood[0])

LabelEncoder()

In [10]:
df["Neighborhood"]=le.transform(df["Neighborhood"])

Let's load our pickled imputer model. Keep in mind that due to its size, I can only store it locally so you won't have access to it. You can run the Demographic_imputation_MICE.ipynb file to generate the model and it should be exactly the same.


In [11]:
import pickle

In [12]:
impute = pickle.load(open("MICE_MODEL.sav", "rb"))


In [13]:
completed_data = impute.impute_new_data(new_data=df)

New missing demographic data has been imputed using the same parameters from the training data.
Now we need to apply the same methodology of taking the average per missing neighborhood.


In [21]:
completed_data_1 = completed_data.complete_data(0)

In [15]:
df_NA = df[df.isna().any(axis=1)]

In [16]:
missing_neigh = list(df_NA["Neighborhood"].unique())

In [17]:
temp = df.dropna()

In [18]:
temp[temp["Neighborhood"].isin(missing_neigh)]

,Description,Post,District,Neighborhood,Longitude,Latitude,Premise,Month,Hour,Year,...,perc18_24,perc25_64,perc65up,perc_asian,perc_aa,perc_hisp,perc_white,median_price_homes_sold,racial_diversity_index,num_households


In [19]:
missing_neigh

[50, 161, 143, 255, 192, 51, 115, 229, 189, 236, 235, 162]

In [22]:
means = completed_data_1.groupby("Neighborhood").mean()

In [23]:
means.loc[0] #gives the mean values for Neighborhood 0

Description                      6.449612
Post                           512.883721
District                         3.000000
Longitude                       39.326003
Latitude                       -76.610587
Premise                         56.945736
Month                            7.131783
Hour                            14.209302
Year                          2017.697674
Outside                          0.457364
Season_autumn                    0.310078
Season_spring                    0.224806
Season_summer                    0.240310
Season_winter                    0.224806
Weapon_FIREARM                   0.116279
Weapon_HANDS                     0.015504
Weapon_KNIFE                     0.062016
Weapon_NONE                      0.705426
Weapon_OTHER                     0.093023
Holiday                          0.054264
Weekend                          0.255814
median_household_income      41717.170689
households_below_poverty        18.261209
perc18_24                       29

In [24]:
means_1 = means[["median_household_income","households_below_poverty","perc18_24","perc25_64","perc65up","perc_asian","perc_aa","perc_hisp","perc_white","median_price_homes_sold","racial_diversity_index","num_households"]]

In [25]:
means_1.loc[missing_neigh]

,median_household_income,households_below_poverty,perc18_24,perc25_64,perc65up,perc_asian,perc_aa,perc_hisp,perc_white,median_price_homes_sold,racial_diversity_index,num_households
Neighborhood,,,,,,,,,,,,
50,39667.630752,18.552965,8.723770,54.941721,10.003844,1.540094,53.824805,10.437545,32.778371,90677.514329,64.105443,97169.607843
161,37655.022367,22.759053,7.994984,55.744814,9.303305,1.841590,40.417766,12.738711,41.200339,79729.898048,68.408272,77835.032538
143,39302.949230,21.276776,8.563042,56.710589,9.147122,2.239167,52.048871,9.753833,35.372410,89208.333333,63.440295,80230.952381
255,37796.117064,22.213774,8.515442,56.638802,8.547951,2.218322,50.795114,9.648152,36.632183,82748.739566,64.327034,72463.681135
192,39533.754094,19.128183,8.304127,55.305136,10.344631,1.792258,53.074666,11.031319,34.001753,82987.118110,65.191030,89140.944882
51,39336.822646,19.292958,8.286141,55.232568,10.589232,1.802811,54.209064,9.837320,32.832776,82451.662791,63.919140,90937.209302
115,39916.039567,18.709303,8.320370,55.303422,10.473837,1.790862,52.818113,10.617414,33.934618,84712.323529,64.625704,90250.000000
229,38973.590962,19.630507,8.256106,55.209266,11.125595,1.866278,53.540003,9.624956,33.317050,80264.634146,63.834691,90902.439024
189,40145.248284,18.336664,8.821019,54.924880,9.645817,1.483072,53.724938,11.118117,32.548303,89086.903226,64.900157,97932.258065


In [26]:
missing_means = means_1.loc[missing_neigh]


In [27]:
missing_means["median_household_income"].loc[161]

37655.02236729482

In [28]:
completed_data_1.loc[completed_data_1["Neighborhood"]==161,"median_household_income"] = missing_means["median_household_income"].loc[161]

In [29]:
completed_data_1.loc[completed_data_1["Neighborhood"]==161]["median_household_income"]

38       37655.022367
68       37655.022367
73       37655.022367
107      37655.022367
125      37655.022367
             ...     
49616    37655.022367
50145    37655.022367
50338    37655.022367
50496    37655.022367
50971    37655.022367
Name: median_household_income, Length: 461, dtype: float64

In [30]:

for i in missing_neigh:
    completed_data_1.loc[completed_data_1["Neighborhood"]==i,"median_household_income"] = missing_means["median_household_income"].loc[i]
    completed_data_1.loc[completed_data_1["Neighborhood"]==i,"households_below_poverty"] = missing_means["households_below_poverty"].loc[i]
    completed_data_1.loc[completed_data_1["Neighborhood"]==i,"perc18_24"] = missing_means["perc18_24"].loc[i]
    completed_data_1.loc[completed_data_1["Neighborhood"]==i,"perc25_64"] = missing_means["perc25_64"].loc[i]
    completed_data_1.loc[completed_data_1["Neighborhood"]==i,"perc65up"] = missing_means["perc65up"].loc[i]
    completed_data_1.loc[completed_data_1["Neighborhood"]==i,"perc_asian"] = missing_means["perc_asian"].loc[i]
    completed_data_1.loc[completed_data_1["Neighborhood"]==i,"perc_aa"] = missing_means["perc_aa"].loc[i]
    completed_data_1.loc[completed_data_1["Neighborhood"]==i,"perc_hisp"] = missing_means["perc_hisp"].loc[i]
    completed_data_1.loc[completed_data_1["Neighborhood"]==i,"perc_white"] = missing_means["perc_white"].loc[i]
    completed_data_1.loc[completed_data_1["Neighborhood"]==i,"median_price_homes_sold"] = missing_means["median_price_homes_sold"].loc[i]
    completed_data_1.loc[completed_data_1["Neighborhood"]==i,"racial_diversity_index"] = missing_means["racial_diversity_index"].loc[i]
    completed_data_1.loc[completed_data_1["Neighborhood"]==i,"num_households"] = missing_means["num_households"].loc[i]

In [32]:
completed_data_1.loc[completed_data_1["Neighborhood"]==255]["median_household_income"]

108      37796.117064
268      37796.117064
306      37796.117064
324      37796.117064
330      37796.117064
             ...     
50661    37796.117064
50916    37796.117064
50934    37796.117064
51040    37796.117064
51045    37796.117064
Name: median_household_income, Length: 599, dtype: float64

In [33]:
completed_data_1.to_csv("../data/TEST_DATA_WITH_MICE_IMPUTED_DEMOGRAPHICS.csv")

In [34]:
completed_data_1.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 51119 entries, 0 to 51118
Data columns (total 34 columns):
 #   Column                    Non-Null Count  Dtype  
---  ------                    --------------  -----  
 0   Description               51119 non-null  int64  
 1   Post                      51119 non-null  float64
 2   District                  51119 non-null  int64  
 3   Neighborhood              51119 non-null  int32  
 4   Longitude                 51119 non-null  float64
 5   Latitude                  51119 non-null  float64
 6   Premise                   51119 non-null  int64  
 7   Month                     51119 non-null  int64  
 8   Hour                      51119 non-null  int64  
 9   Year                      51119 non-null  int64  
 10  Outside                   51119 non-null  int64  
 11  Season_autumn             51119 non-null  int64  
 12  Season_spring             51119 non-null  int64  
 13  Season_summer             51119 non-null  int64  
 14  Season